# Text segmentation using Hidden Markov Models

### Task : automatic segmentation of mails, problem statement

In [1]:
import numpy as np

## Q1 :
**Give the value of the $\pi$ vector of the initial probabilities**

$\pi = \{ 1, 0 \}$

In [2]:
pi = np.array([1, 0])

In [146]:
# transition matrix 
A = np.array([
              [0.999218078035812 ,0.000781921964187974],
              [0                 ,1                   ]
             ])

### Q2 :

**- What is the probability to move from state 1 to state 2 ?**

**- What is the probability to remain in state 2 ?**

**- What is the lower/higher probability ? Try to explain why**

- P (move from state 1 to 2) $= \mathbb{P}(q2 | q1) = \mathbb{P}(2 | 1) = A(1, 2) = 0.000781921964187974$

- P (stay in state 2) = P (move from state 2 to 2) $= \mathbb{P}(q2 | q2) = \mathbb{P}(2 | 2) = A(2, 2) = 1$

  It is deterministic to stay at state 2, once we are in state 2. This result is logical since once we are in the body we can't get back to the header.
  
  The probability for the text to move from the header to the body is very low. Actually, this event only happens once in a document, and since there are many words in a text, the probability to jump to the body is low, but not null.

- The lowest probability, which is 0, is the prob to go from state 2 to state 1, which is from body to header. This is impossible since there is no going back.
  
  The higher probability was explained already; we are sure that we won't change the state again to header once we have reached the body, so it is always the other way around.

In [145]:
A[0,1], A[1,1]

(0.000781921964187974, 1.0)

## Q3 :
**What is the size of B ?**

B is the emission (observation) probabilities. We have multiple files, each containing a text of N characters. 

Each of these N charcters is one of the 256 chars encoded on 1 ascii byte. So, B represents the probabilities in each state for each of the 256 characters to appear.

Hence, the shape of B is 256x2.

### Material
##### Coding/decoding mails


In [227]:
texts = []
names = open('./dat/mail.lst').read().splitlines()
for i in range(len(names)):
    print(names[i])
    names[i] = './dat/' + names[i]
    texts.append(np.loadtxt(names[i], dtype=int))    

mail1.dat
mail10.dat
mail11.dat
mail12.dat
mail13.dat
mail14.dat
mail15.dat
mail16.dat
mail17.dat
mail18.dat
mail19.dat
mail2.dat
mail20.dat
mail21.dat
mail22.dat
mail23.dat
mail24.dat
mail25.dat
mail26.dat
mail27.dat
mail28.dat
mail29.dat
mail3.dat
mail30.dat
mail4.dat
mail5.dat
mail6.dat
mail7.dat
mail8.dat
mail9.dat


In [230]:
texts[0][:100]

array([ 70, 114, 111, 109,  32, 101, 120, 109, 104,  45, 119, 111, 114,
       107, 101, 114, 115,  45,  97, 100, 109, 105, 110,  64, 114, 101,
       100, 104,  97, 116,  46,  99, 111, 109,  32,  32,  84, 104, 117,
        32,  65, 117, 103,  32,  50,  50,  32,  49,  50,  58,  51,  54,
        58,  50,  51,  32,  50,  48,  48,  50,  10,  82, 101, 116, 117,
       114, 110,  45,  80,  97, 116, 104,  58,  32,  60, 101, 120, 109,
       104,  45, 119, 111, 114, 107, 101, 114, 115,  45,  97, 100, 109,
       105, 110,  64, 115, 112,  97, 109,  97, 115])

In [209]:
mails = dict(zip(names, texts))

In [211]:
names[0], texts[0]

('./dat/mail1.dat', array([ 70, 114, 111, ..., 115,  10,  10]))

In [238]:
texts

[array([ 70, 114, 111, ..., 115,  10,  10]),
 array([ 70, 114, 111, ..., 107,  10,  10]),
 array([ 70, 114, 111, ..., 108,  10,  10]),
 array([ 70, 114, 111, ..., 108,  10,  10]),
 array([ 70, 114, 111, ..., 101,  10,  10]),
 array([ 70, 114, 111, ..., 115,  10,  10]),
 array([ 70, 114, 111, ..., 107,  10,  10]),
 array([ 70, 114, 111, ..., 117,  10,  10]),
 array([ 70, 114, 111, ...,  10,  10,  10]),
 array([ 70, 114, 111, ..., 101,  10,  10]),
 array([ 70, 114, 111, ...,  10,  10,  10]),
 array([ 70, 114, 111, ...,  10,  10,  10]),
 array([ 70, 114, 111, ..., 101,  10,  10]),
 array([ 70, 114, 111, ...,  10,  10,  10]),
 array([ 70, 114, 111, ..., 101,  10,  10]),
 array([ 70, 114, 111, ..., 101,  10,  10]),
 array([ 70, 114, 111, ...,  10,  10,  10]),
 array([ 70, 114, 111, ..., 101,  10,  10]),
 array([ 70, 114, 111, ..., 107,  10,  10]),
 array([ 70, 114, 111, ..., 101,  10,  10]),
 array([ 70, 114, 111, ..., 107,  10,  10]),
 array([ 70, 114, 111, ..., 107,  10,  10]),
 array([ 7

##### Distribution files

In [143]:
P = np.loadtxt('./PerlScriptAndModel/P.text')

In [144]:
P.shape, np.sum(P, axis=0)

((256, 2), array([1.00000039, 1.00000061]))

The P matrix gives us the distribution probability of all the 255 ascii characters, between both states 1 and 2 (header and body).


In [64]:
chr(P[:,0].argmax()), chr(P[:,1].argmax())  

(' ', ' ')

The most probable characters while in header or in body (in all the text) is the space character, which is obvious.

In [133]:
print("char        header            body\n")  #ord('a')
for i, (hed, bod) in enumerate(P):
    if hed + bod > 1e-5:
        if i == 9:
            print(" ", "\\t", "   ", "{:1.6}         {:1.6}".format(hed, bod))
        elif i == 10:
            print(" ", "\\n", "   ", "{:1.6}          {:1.6}".format(hed, bod))
        elif i == 32:
            print(" ", "' '", "   ", "{:1.6}          {:1.6}".format(hed, bod))
        else:
            print(" ", chr(i), "    ", "{:1.6}        {:1.6}".format(hed, bod))

char        header            body

  \t     0.00156408         6.12724e-07
  \n     0.0188428          0.0256125
  ' '     0.089717          0.148709
  !      3.90922e-07        0.000858427
  "      0.000625866        0.00177751
  #      0.000117668        6.12724e-07
  $      7.85753e-05        0.00018443
  &      3.90922e-07        0.00042952
  '      0.000117668        0.00171624
  (      0.00723245        0.000552065
  )      0.00723245        0.000552065
  +      0.00175954        0.000552065
  ,      0.00277594        0.00814985
  -      0.0185301        0.0526949
  .      0.0443309        0.0110297
  /      0.00332323        0.00667931
  0      0.0369425        0.00404459
  1      0.0232993        0.00239024
  2      0.0301014        0.00263533
  3      0.00883523        0.00116479
  4      0.00914797        0.0015937
  5      0.00828794        0.000858427
  6      0.010008        0.000858427
  7      0.0064506        0.00128733
  8      0.00754519        0.000919699
  9      0

### Viterbi algorithm

In [29]:
# !/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Jun  6 10:03:27 2018

@author: chloeclavel
"""
def viterbi_obsolete(X, A, B, pi, states):
    """
        Viterbi Algorithm Implementation

        Keyword arguments:
            - X: sequence of observation (dim: 1xN)
            - A: transition matrix (dim: QxQ)
            - B: emission probability matrix (dim: NxQ)
            - pi: initial vector of probabilities (dim: 1xQ)
            - states:list of states (dim: 1xQ)
        Returns:
            - seq: sequence of state
    """

#     realmin = np.finfo(np.double).tiny  # smallest 'visible' number
#     A = np.log(A+realmin)               # avoids log(0)
#     B = np.log(B+realmin)               # avoids log(0)
#     pi = np.log(pi+realmin)             # avoids log(0)
    
    T = X.shape[0]                      # number of observed characters
    N = B.shape[0]                      # number of characters
    Q = pi.shape[0]                     # model's number of states
    print("dict length:", N, ", observed characters:", T, ", number of states:", Q)
    
    #Initialisation
    logl = np.zeros((Q,T))
    bcktr = np.zeros((N-1,Q))
    
    logl[:,0] = pi * B[X[0],:]       # initial delta
    for t in range(1,(T-1)):
        for q in range(Q):
            state = logl[:,t-1] * 1
        logl[:,t] = logl[:,t-1] + B[X[t],:]
    
    path = 1
    return logl, path


In [337]:
# !/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Jun  6 10:03:27 2018

@author: bourhandernayka
"""
def viterbi_prod(X, A, B, pi, states):
    """
        Viterbi Algorithm Implementation

        Keyword arguments:
            - X: sequence of observation (dim: 1xN)
            - A: transition matrix (dim: QxQ)
            - B: emission probability matrix (dim: NxQ)
            - pi: initial vector of probabilities (dim: 1xQ)
            - states:list of states (dim: 1xQ)
        Returns:
            - seq: sequence of state
    """

#     realmin = np.finfo(np.double).tiny  # smallest 'visible' number
#     A = np.log(A+realmin)               # avoids log(0)
#     B = np.log(B+realmin)               # avoids log(0)
#     pi = np.log(pi+realmin)             # avoids log(0)
    
    T = X.shape[0]                      # number of observed characters
    N = B.shape[0]                      # number of characters
    Q = pi.shape[0]                     # model's number of states
    print("dict length:", N, ", observed characters:", T, ", number of states:", Q)
    
    #Initialisation
    delta = np.zeros((Q,T))
    bcktr = -1*np.ones((Q,T), dtype=int)
    path = -1*np.ones(T, dtype=int)
    
    delta[:,0] = pi * B[X[0],:]       # initial delta (dim: Qx1)
    for t in range(1, T):
        for q in range(Q):
            delta[q,t] = np.max(delta[:,t-1] * A[:, q]) *  B[X[t],q]
            bcktr[q,t] = np.argmax(delta[:,t-1] * A[:, q])
    
    path[T-1] = states[bcktr[states[np.argmax(delta[:,T-1])], T-1]]
    for t in range(T-1, 0, -1):
        path[t-1] = states[bcktr[path[t], t]]
    
    return delta, bcktr, path


In [305]:
# !/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Jun  6 10:03:27 2018

@author: bourhandernayka
"""
def viterbi(X, A, B, pi, states):
    """
        Viterbi Algorithm Implementation

        Keyword arguments:
            - X: sequence of observation (dim: 1xN)
            - A: transition matrix (dim: QxQ)
            - B: emission probability matrix (dim: NxQ)
            - pi: initial vector of probabilities (dim: 1xQ)
            - states: list of states (dim: 1xQ)
        Returns:
            - path: sequence of predicted states
    """

    realmin = np.finfo(np.double).tiny  # smallest 'visible' number
    A = np.log(A+realmin)               # avoids log(0)
    B = np.log(B+realmin)               # avoids log(0)
    pi = np.log(pi+realmin)             # avoids log(0)
    
    T = X.shape[0]                      # number of observed characters
    N = B.shape[0]                      # number of characters
    Q = pi.shape[0]                     # model's number of states
    print("dict length:", N, ", observed characters:", T, ", number of states:", Q)
    
    #Initialisation
    delta = np.zeros((Q,T))
    bcktr = -1*np.ones((Q,T), dtype=int)
    path  = -1*np.ones(T, dtype=int)
    
    # initial delta (dim: Qx1)
    delta[:,0] = pi + B[X[0],:]
    
    # recursion
    for t in range(1, T):
        for q in range(Q):
            delta[q,t] = np.max(delta[:,t-1] + A[:, q]) +  B[X[t],q]
            bcktr[q,t] = states[np.argmax((delta[:,t-1] + A[:, q]))]
    
    # backtrack
    path[T-1] = states[bcktr[states[np.argmax(delta[:,T-1])], T-1]]
    for t in range(T-1, 0, -1):
        path[t-1] = states[bcktr[path[t], t]]

    return delta, bcktr, path

___
#### Implementing the example from the course to check results:

In [338]:
viterbi_prod(np.array([0,0,1,1]), np.array([[.3,.5,.2],
                                       [0, .3, .7],
                                       [0, 0, 1]
                                      ]),
       np.array([[1, .5, 0], 
                 [0, .5, 1]]),
       np.array([0.6, .4, 0]),
       np.array([0, 1, 2], dtype=int))

dict length: 2 , observed characters: 4 , number of states: 3


(array([[0.6    , 0.18   , 0.     , 0.     ],
        [0.2    , 0.15   , 0.045  , 0.00675],
        [0.     , 0.     , 0.105  , 0.105  ]]),
 array([[-1,  0,  0,  0],
        [-1,  0,  0,  1],
        [-1,  1,  1,  2]]),
 array([0, 1, 2, 2]))

In [307]:
viterbi(np.array([0,0,1,1]), np.array([[.3,.5,.2],
                                       [0, .3, .7],
                                       [0, 0, 1]
                                      ]),
       np.array([[1, .5, 0], 
                 [0, .5, 1]]),
       np.array([0.6, .4, 0]),
       np.array([0, 1, 2], dtype=int))

dict length: 2 , observed characters: 4 , number of states: 3


(array([[-5.10825624e-01, -1.71479843e+00, -7.11315190e+02,
         -1.41904663e+03],
        [-1.60943791e+00, -1.89711998e+00, -3.10109279e+00,
         -4.99821277e+00],
        [-1.41679284e+03, -7.10362531e+02, -2.25379493e+00,
         -2.25379493e+00]]),
 array([[-1,  0,  0,  2],
        [-1,  0,  0,  1],
        [-1,  1,  1,  2]]),
 array([0, 1, 2, 2]))

This matches exactly the results from the course, for the both algorithms (with and without log).
___

In [308]:
# other experiment:
viterbi_prod(np.array([1, 4, 6, 5, 2, 0, 3, 3, 3]), np.array([[.993, .007],[0,1]]),
       np.array([[.2, .9, .5, .1, .7, .5, .6], 
                 [.8, .1, .5, .9, .3, .5, .4]]).T,
       np.array([1, 0]),
       [0,1])

dict length: 7 , observed characters: 9 , number of states: 2


(array([[9.00000000e-01, 6.25590000e-01, 3.72726522e-01, 1.85058718e-01,
         9.18816536e-02, 1.82476964e-02, 1.81199625e-03, 1.79931228e-04,
         1.78671709e-05],
        [0.00000000e+00, 1.89000000e-03, 1.75165200e-03, 1.30454283e-03,
         6.52271413e-04, 5.21817131e-04, 4.69635418e-04, 4.22671876e-04,
         3.80404688e-04]]),
 array([[-1,  0,  0,  0,  0,  0,  0,  0,  0],
        [-1,  0,  0,  0,  1,  1,  1,  1,  1]]),
 array([0., 0., 0., 0., 0., 0., 1., 1., 1.]))

___
Now, we get back to test our Viterbi on our mails:

In [339]:
# testing on the first mail:
deltas, track, path = viterbi(texts[0], A, P, pi, [0,1])

dict length: 256 , observed characters: 5216 , number of states: 2


In [340]:
# checking where to cut, and if there were more than one spot
for i in range(len(path)-1):
    if path[i] == 0 and path[i+1] == 1:
        print(i)

3795


The mail will be cut in just one spot, which is reasonable. This means that the labels only alternate once, jumping from 0 to 1, after 3795 characters read.

#### Visualizing Segmentation

In [341]:
def visualize_segmentation(mail_filename, visualized_mail_filename, path):
    ## @parameter mail_filename : Path to the mail on wich we try the algorithm.
    ## @parameter visualized_mail_filename : The path on which we write the mail with the v
    ## @parameter path : The sequence of 0 and 1 that the Viterbi algorithm returns.
    ## return: True if the header corresponds to the
    i = 0
    while path[i] == 0:
        i+=1
    print('cutting at char', i)
    visu = open(visualized_mail_filename, 'w') 
    if(mail_filename.endswith(".dat")):
        mail_filename = mail_filename[:-4] + ".txt" 
    mail = open(mail_filename, 'r')
    header = mail.read(i)
    visu.write(header) 
    visu.write("\n===================== cut here =======================\n") 
    visu.write(mail.read(int(path.sum())))
    visu.close() 
    mail.close() 

In [342]:
visualize_segmentation(names[0], 'out_now.txt', path)

cutting at char 3796


The output file shows the cut, where it fits exactly the separation between the header and the body.

## Q4 :
**Print the track then present and discuss the results obtained on mail11.txt to mail30.txt**

In [345]:
path, path[3700:3900]

(array([0, 0, 0, ..., 1, 1, 1]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1]))

    Date: Thu, 22 Aug 2002 18:26:25 +0700

        Date:        Wed, 21 Aug 2002 10:54:46 -0500
        From:        Chris Garrigues <cwg-dated-1030377287.06fa6d@DeepEddy.Com>
        Message-ID:  <1029945287.4797.TMDA@deepeddy.vircio.com>
    ===================== cut here =======================



      | I can't reproduce this error.

    For me it is very repeatable... (like every time, without fail).

    This is the debug log of the pick happening ...

We remark that the cut happens at a logical place, just after the non-subject text part, and just after the cut the actaual body of the message begins. This extract is that of the mail1.txt

#### Automatizing the segmentation process

In [330]:
def split_mail(mailname):
    ### This function generalizes the mail treatment procedure, by taking into input just the name of the mail text file.
    nb = mailname[-6:-4]
    deltas, track, path = viterbi(mails[mailname], A, P, pi, [0, 1])
    visualize_segmentation(mailname, 'out'+nb+'.txt', path)
    
    return path

In [346]:
path = split_mail('./dat/mail11.dat')

dict length: 256 , observed characters: 3475 , number of states: 2
cutting at char 2851


**For the mail11.txt:**

    List-Archive: <http://www.geocrawler.com/redir-sf.php3?list=spamassassin-devel>
    X-Original-Date: Thu, 22 Aug 2002 16:19:48 +0200
    Date: Thu, 22 Aug 2002 16:19:48 +0200
    ===================== cut here =======================


    Hello, have you seen and discussed this article and his approach?

    Thank you

Once again, the cut is perfect, just at the beginning of the actual subject of the mail

In [347]:
path = split_mail('./dat/mail21.dat')

dict length: 256 , observed characters: 2664 , number of states: 2
cutting at char 2103


    Date: Thu, 22 Aug 2002 17:23:28 +0100
    Subject: Re: [zzzzteana] Which Muppet Are You?
    Reply-To: zzzzteana@yahoogroups.com
    Content-Type: text/plain; charset=US-ASCII
    C
    ===================== cut here =======================
    ontent-Transfer-Encoding: 7bit

    > Apols if this has been posted before:
    > 
    > http://www.pinkpaperclips.net/subs/quiz2.html
    >
    So, anyone who isn't Beaker?

    TimC
    Meep

This time, the cut was mislead by 1 line, however it is still relevant as a cut.

In [349]:
path = split_mail('./dat/mail29.dat')

dict length: 256 , observed characters: 2890 , number of states: 2
cutting at char 2344


Mail29 was also cut finely, at the perfect spot.

#### Further questions:
## Q5
**How would you model the problem if you had to segment the mails in more than two parts
(for example : header, body, signature) ?**

If we had more than 2 parts, we will have to increase the dimension of the transition matrix and that of the initial vector states. Also, the observations probabilities will be then distributed over all the possible states...

In the example of 3 possible segments, the transition matrix A will be of dim 3x3, the initial states vector would be of dim 3x1, and the observations probabilities matrix will be of dim 26x3, each column corresponding to header, body and signature, respectively.

However, ecverything else would be unchanged, even the Viterbi algorithm. This is very generic.

## Q6
**How would you model the problem of separating the portions of mail included, knowing that
they always start with the character ">" ?**

Knowing that new segments always start with ">", we would increase the probability of the observation of ">" leading to a more probable jump to the next state once we encounter ">".

In fact, there would be no change in terms of the transition matrx, nor the initial probabilities matrix. Just the B matrix will change at the row of the observation ">".